<a href="https://colab.research.google.com/github/OctavaOrigin/Classic-Machine-Learning/blob/main/%D0%9F%D1%80%D0%BE%D0%B1%D0%BB%D0%B5%D0%BC%D0%B0_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data_full = pd.read_csv('train.csv')

In [ ]:
data_full.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
data = data_full.loc[:, ~data_full.columns.isin(['Name', 'PassengerId', 'Sex', 'Ticket', 'Cabin', 'Embarked'])]
data = data.loc[data.isna().any(axis=1) == False]
y = data['Survived']
X = data.loc[:, ~data.columns.isin(['Survived'])]

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X, y)

LogisticRegression()

In [ ]:
lr.score(X, y)

0.7002801120448179

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score

In [ ]:
y_pred = lr.predict(X)

In [ ]:
roc_auc_score(y, y_pred)

0.6713565387117761

In [ ]:
f1_score(y, y_pred)

0.5836575875486382

Теперь поработаем с данными

In [ ]:
data_full.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Cabin вроде бы полезная, но там слишком много нулей, избавимся от неё. Также сразу удалим номер пассажира, его имя и билет

In [ ]:
data = data_full.loc[:, ~data_full.columns.isin(['PassengerId', 'Name', 'Cabin', 'Ticket'])]

In [ ]:
data.loc[data['Embarked'].isna()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,1,female,38.0,0,0,80.0,NaN
829,1,1,female,62.0,0,0,80.0,NaN


Эти удалим

In [ ]:
data = data.loc[data['Embarked'].isna() == False]

In [ ]:
data.loc[data['Age'].isna()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,0,3,male,NaN,0,0,8.4583,Q
17,1,2,male,NaN,0,0,13.0000,S
19,1,3,female,NaN,0,0,7.2250,C
26,0,3,male,NaN,0,0,7.2250,C
28,1,3,female,NaN,0,0,7.8792,Q
...,...,...,...,...,...,...,...,...
859,0,3,male,NaN,0,0,7.2292,C
863,0,3,female,NaN,8,2,69.5500,S
868,0,3,male,NaN,0,0,9.5000,S
878,0,3,male,NaN,0,0,7.8958,S


Вот эти заполним с помощью линейной регрессии

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
df = data.copy()
encoder = LabelEncoder()
model = LinearRegression()
poly_features = PolynomialFeatures(degree=1)

df['Sex'] = encoder.fit_transform(df['Sex'])
df = pd.get_dummies(df, columns=['Embarked'])

def remove_outliers(data, column, multiplier=1.5):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - multiplier * iqr
    upper_bound = q3 + multiplier * iqr
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

cols = ['Age', 'Fare']
for col in cols:
  df = remove_outliers(df, col)

df = df.loc[:, ~df.columns.isin(['SibSp', 'Parch', 'Embarked_S'])]

y = df['Age']
X = df
del df['Age']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size=0.2)

X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.fit_transform(X_test)
model.fit(X_train_poly, y_train)
y_pred = model.predict(X_test_poly)

r2_score(y_test, y_pred)

0.26805579797792334

Так себе модель, но лучше не получилось

In [ ]:
df = data.loc[data['Age'].isna()]

df['Sex'] = encoder.fit_transform(df['Sex'])
df = pd.get_dummies(df, columns=['Embarked'])

df = df.loc[:, ~df.columns.isin(['SibSp', 'Parch', 'Embarked_S'])]

del df['Age']
X = poly_features.fit_transform(df)
df['Age'] = model.predict(X)

<ipython-input-408-c512b4306b63>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sex'] = encoder.fit_transform(df['Sex'])


In [ ]:
df['Age'].values
data.loc[data['Age'].isna(), ['Age']] = df['Age'].values

Вот что сделаем. Избавимся от Parch и SibSp (всё равно там одни нули), разложим Embarked и уберём выбросы из Age и Fare

In [ ]:
del data['SibSp']
del data['Parch']

In [ ]:
cols = ['Age', 'Fare']
for col in cols:
  data = remove_outliers(data, col)

In [ ]:
data = pd.get_dummies(data, columns=['Embarked'])
data['Sex'] = encoder.fit_transform(data['Sex'])

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
y = data['Survived']
del data['Survived']
X = data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8410596026490066

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
roc_auc_score(y_test, y_pred)

0.8104950495049505

In [ ]:
f1_score(y_test, y_pred)

0.7499999999999999

**старый** f1 = 0.5836
**новый** f1 = 0.75

**старый** roc-auc = 0.6713
**новый** roc-auc = 0.8104

**старый** accuracy = 0.7002
**новый** accuracy = 0.84105